In [1]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, data_home='../data',cache=True,as_frame=False)
X=X/255.

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [45]:
num_examples,D=X_train.shape
num_outputs=len(np.unique(y_train))

model_ctx=mx.gpu()
net = gluon.nn.Dense(num_outputs)

In [46]:
net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=model_ctx)

In [47]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

In [48]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [49]:
def iterate_minibatches(X, y, batchsize,ctx):
    assert X.shape[0] == y.shape[0]
    for start_idx in range(0, X.shape[0] - batchsize + 1, batchsize):
        excerpt = slice(start_idx, start_idx + batchsize)
        yield nd.array(X[excerpt],ctx=ctx), nd.array(y[excerpt],ctx=ctx)


In [50]:
def evaluate_accuracy(net,X,y,batch_size,ctx):
    acc = mx.metric.Accuracy()
    for data,label in iterate_minibatches(X,y,batch_size,ctx):
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [51]:
epochs = 10
moving_loss = 0.
batch_size=64
for e in range(epochs):
    cumulative_loss = 0
    for data,label in iterate_minibatches(X,y,batch_size,model_ctx):
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        cumulative_loss += nd.sum(loss).asscalar()

    test_accuracy = evaluate_accuracy(net,X_train,y_train,batch_size,model_ctx)
    train_accuracy = evaluate_accuracy(net,X_test,y_test,batch_size,model_ctx)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, cumulative_loss/num_examples, train_accuracy, test_accuracy))

Epoch 0. Loss: 2.542672055943736, Train_acc 0.8008170871559633, Test_acc 0.8016071428571429
Epoch 1. Loss: 1.0615867886330401, Train_acc 0.8419581422018348, Test_acc 0.8411964285714286
Epoch 2. Loss: 0.8728184244201652, Train_acc 0.857368119266055, Test_acc 0.8571785714285715
Epoch 3. Loss: 0.774513258108603, Train_acc 0.8672591743119266, Test_acc 0.8669464285714286
Epoch 4. Loss: 0.7108107527095292, Train_acc 0.8734948394495413, Test_acc 0.8737857142857143
Epoch 5. Loss: 0.664924758946257, Train_acc 0.8777952981651376, Test_acc 0.8785535714285714
Epoch 6. Loss: 0.629691659716091, Train_acc 0.8823824541284404, Test_acc 0.8823928571428571
Epoch 7. Loss: 0.6014451960735023, Train_acc 0.8843893348623854, Test_acc 0.88525
Epoch 8. Loss: 0.5780878436389778, Train_acc 0.8872563073394495, Test_acc 0.8880892857142857
Epoch 9. Loss: 0.558328430362578, Train_acc 0.8901232798165137, Test_acc 0.8901071428571429


In [57]:
for par in net.params:
    print(par)

dense2_weight
dense2_bias


In [42]:
y_pred=net(nd.array(X_test,ctx=model_ctx))

In [43]:
from sklearn.metrics import classification_report

print(classification_report(np.int32(y_test), y_pred.argmax(axis=1).asnumpy()))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96      1367
           1       0.95      0.98      0.96      1579
           2       0.89      0.91      0.90      1396
           3       0.87      0.91      0.89      1448
           4       0.93      0.91      0.92      1346
           5       0.89      0.86      0.88      1273
           6       0.96      0.95      0.95      1416
           7       0.93      0.93      0.93      1483
           8       0.90      0.83      0.87      1303
           9       0.87      0.91      0.89      1389

    accuracy                           0.92     14000
   macro avg       0.92      0.91      0.91     14000
weighted avg       0.92      0.92      0.92     14000

